# Circos plot of cancer genomics 

This notebook follows closely to [this online tutorial](https://training.galaxyproject.org/training-material/topics/visualisation/tutorials/circos/tutorial.html#example-cancer-genomics).  The datasets are available [here](https://zenodo.org/record/4494146/#.Y0cinnbMIQg) and should be downloaded into the ```data``` directory (which you may have to create).

We will use the [pyCircos](https://github.com/ponnhide/pyCircos) package in Python. 

*Note:* this Jupyter notebook was converted to an html file using the following command in a terminal :

```
jupyter nbconvert --execute --to html circos.ipynb
```
___
*Author : Aaron M Geller, Dec. 2022*

In [ ]:
# Run this cell to create the data directory and download the sample data.
# On subsequent runs of this notebook, you can skip this step
# Note: the first two files here are large (~40MB each). So, this cell may take a while to run.
# If they don't download properly here, you can simply download them manually from the 
#   zenodo website: https://zenodo.org/record/4494146/

directory = file.path(getwd(), 'data')
dir.create(directory)

download.file('https://zenodo.org/record/4494146/files/VCaP_Copy-Number.tsv?download=1', 
    file.path(directory, 'VCaP_Copy-Number.tsv'))
download.file('https://zenodo.org/record/4494146/files/VCaP_B-allele-Frequency.tsv?download=1', 
    file.path(directory, 'VCaP_B-allele-Frequency.tsv'))
download.file('https://zenodo.org/record/4494146/files/VCaP-highConfidenceJunctions.tsv?download=1', 
    file.path(directory, 'VCaP-highConfidenceJunctions.tsv'))
download.file('https://zenodo.org/record/4494146/files/hg18_karyotype_withbands.txt?download=1', 
    file.path(directory, 'hg18_karyotype_withbands.txt'))

In [ ]:
library(RCircos)
library(stringr)
library(data.table)

## 1. Read in the Karyotype Configuration data using [`read.table`](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/read.table).

In [ ]:
# read in the data and define the columns
# skip extra rows at the top that are not useful here
dfChrom <- read.table('data/hg18_karyotype_withbands.txt', sep = "" , header = FALSE, skip = 24)
colnames(dfChrom) <- c('','chromosome','band','','chromStart','chromEnd','Stain')
head(dfChrom)

## 1.1 Modify the table so that it can be used more easily with RCircos.

In [ ]:
# reorder to columns for RCircos (and drop a few that are unnecessary)
dfChrom <- dfChrom[, c(2, 5, 6, 3, 7)]

# create a column with the chromosome number or string for sorting
dfChrom$chromosomeID <- gsub("chr","",as.character(dfChrom$chromosome))

# sort by chromosomeID
dfChrom <- dfChrom[str_order(dfChrom$chromosomeID, numeric = TRUE),]

head(dfChrom)

## 1.2 Create the initial circos figure showing the Ideogram.

In [ ]:
# for displaying in Jupyter
options(jupyter.plot_mimetypes = "image/png", repr.plot.width = 6, repr.plot.height = 6, repr.plot.res = 300)

# add the data to RCircos
RCircos.Set.Core.Components(cyto.info = dfChrom, chr.exclude = NULL, tracks.inside = 4, tracks.outside = 0)
RCircos.Set.Plot.Area()
RCircos.Chromosome.Ideogram.Plot() 

## 2. Read in the Copy Number data using [`read.table`](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/read.table).

In [ ]:
# read in the copy number variation (CNV) data and fix the column names
dfCNV <- read.table('data/VCaP_Copy-Number.tsv', header = TRUE)
colnames(dfCNV) <- c('chromosome','start','stop','value','')
head(dfCNV)

## 2.1 Modify the table so that it can be used more easily with RCircos.

In [ ]:
# drop the unnecessary last column
dfCNV <- dfCNV[, c(1, 2, 3, 4)]

# remove any extra chromosomes that are not in the ideogram
extras <- c('chrMT')
dfCNV <- subset(dfCNV, !(chromosome %in% extras))

# create a column with the chromosome number or string for sorting
dfCNV$chromosomeID <- gsub("chr","",as.character(dfCNV$chromosome))

# randomly select 25000 lines (otherwise the plot is too full and takes too long to create)
dfCNV <- dfCNV[sample(nrow(dfCNV), 25000), ]

# sort by chromosomeID
dfCNV <- dfCNV[str_order(dfCNV$chromosomeID, numeric = TRUE),]

# limit the data values so they don't go outside of the plot
dfCNV <- subset(dfCNV, (value >= -1))
dfCNV <- subset(dfCNV, (value <= 1))

head(dfCNV)

## 2.2 Add the Copy Number data to the circos figure as a scatter plot.

Documentation for the scatter plot is available [here](https://www.rdocumentation.org/packages/RCircos/versions/1.2.2/topics/RCircos.Scatter.Plot).

In [ ]:
# for displaying in Jupyter
options(jupyter.plot_mimetypes = "image/png", repr.plot.width = 6, repr.plot.height = 6, repr.plot.res = 300)

# add the data to RCircos
RCircos.Set.Core.Components(cyto.info = dfChrom, chr.exclude = NULL, tracks.inside = 4, tracks.outside = 0)

# modify default parameters by accessing the params object (defined below)
params <- RCircos.Get.Plot.Parameters()

# change the background to white (default is cream)
params$track.background <- "white"

# update the plot parameters
RCircos.Reset.Plot.Parameters(params)

# generate the plots
RCircos.Set.Plot.Area()
RCircos.Chromosome.Ideogram.Plot() 

# for the scatter plot we define the following parameters:
# data.col : the column number for the values that we want to plot
# track.num : the ring in the circos plot where this scatter plot should live
# side : position relative to the ideogram
# by.fold : separation between blue and red points
# min.value : minimum (y) value for the plot axis
# max.value : maximum (y) value for the plot axis
RCircos.Scatter.Plot(dfCNV, data.col = 4, track.num = 1, side = 'in', by.fold = 0.15, min.value = -1, max.value = 1)


## 3. Read in the B-Allele data using [`read.table`](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/read.table).

In [ ]:
# read in the B-allele frequency (BAF) data and fix the column names
dfBAF <- read.table('data/VCaP_B-allele-Frequency.tsv', header = TRUE)
colnames(dfBAF) <- c('chromosome','start','stop','value')
head(dfBAF)

## 3.1 Modify the table so that it can be used more easily with RCircos.

In [ ]:
# remove any extra chromosomes that are not in the ideogram
extras <- c('chrMT')
dfBAF <- subset(dfBAF, !(chromosome %in% extras))

# create a column with the chromosome number or string for sorting
dfBAF$chromosomeID <- gsub("chr","",as.character(dfBAF$chromosome))

# randomly select 25000 lines (otherwise the plot is too full and takes too long to create)
dfBAF <- dfBAF[sample(nrow(dfBAF), 25000), ]

# sort by chromosomeID
dfBAF <- dfBAF[str_order(dfBAF$chromosomeID, numeric = TRUE),]

head(dfBAF)

## 3.2 Add the B-Allele data to the circos figure as a scatter plot.

Documentation for the scatter plot is available [here](https://www.rdocumentation.org/packages/RCircos/versions/1.2.2/topics/RCircos.Scatter.Plot).

In [ ]:
# for displaying in Jupyter
options(jupyter.plot_mimetypes = "image/png", repr.plot.width = 6, repr.plot.height = 6, repr.plot.res = 300)

# add the data to RCircos and set parameters
RCircos.Set.Core.Components(cyto.info = dfChrom, chr.exclude = NULL, tracks.inside = 4, tracks.outside = 0)
params <- RCircos.Get.Plot.Parameters()
params$track.background <- "white"
RCircos.Reset.Plot.Parameters(params)
RCircos.Set.Plot.Area()

# ideogram
RCircos.Chromosome.Ideogram.Plot() 

# scatter plots
RCircos.Scatter.Plot(dfCNV, data.col = 4, track.num = 1, side = 'in', by.fold = 0.15, min.value = -1, max.value = 1)
RCircos.Scatter.Plot(dfBAF, data.col = 4, track.num = 2, side = 'in', min.value = 0, max.value = 1)

## 4. Read in the Junctions data using [`data.table::fread`](https://www.rdocumentation.org/packages/data.table/versions/1.14.6/topics/fread).

Note: we are using fread here because the data table has columns of different lengths

In [ ]:
# read in the junctions data and fix the column names
dfLinks <- fread('data/VCaP-highConfidenceJunctions.tsv', header = TRUE, skip = 11)
head(dfLinks)

## 4.1 Modify the table so that it can be used more easily with RCircos

In [ ]:
# select the necessary columns
dfLinks <- dfLinks[, c(2, 3, 3, 6, 7, 7)]

colnames(dfLinks) <- c('chromosome','chromStart','chromEnd','chromosome.1','chromStart.1','chromEnd.1')

# remove any extra chromosomes that are not in the ideogram
extras <- c('chrMT')
dfLinks <- subset(dfLinks, !(chromosome %in% extras))
dfLinks <- subset(dfLinks, !(chromosome.1 %in% extras))

# only use links that go between different chromosomes
dfLinks <- subset(dfLinks, (chromosome != chromosome.1))

# set the color for each line (in this case all black)
dfLinks$PlotColor <- "black"

head(dfLinks)

## 4.2 Add the Junctions data to the circos figure as a link plot.

Documentation for the link plot is available [here](https://www.rdocumentation.org/packages/RCircos/versions/1.2.2/topics/RCircos.Link.Plot).

In [ ]:
# for displaying in Jupyter
options(jupyter.plot_mimetypes = "image/png", repr.plot.width = 6, repr.plot.height = 6, repr.plot.res = 300)

# add the data to RCircos and set parameters
RCircos.Set.Core.Components(cyto.info = dfChrom, chr.exclude = NULL, tracks.inside = 4, tracks.outside = 0)
params <- RCircos.Get.Plot.Parameters()
params$track.background <- "white"
RCircos.Reset.Plot.Parameters(params)
RCircos.Set.Plot.Area()

# ideogram
RCircos.Chromosome.Ideogram.Plot() 

# scatter plots
RCircos.Scatter.Plot(dfCNV, data.col = 4, track.num = 1, side = 'in', by.fold = 0.15, min.value = -1, max.value = 1)
RCircos.Scatter.Plot(dfBAF, data.col = 4, track.num = 2, side = 'in', min.value = 0, max.value = 1)

# for the links plot we define the following parameters:
# track.num : the ring in the circos plot where this link plot should live
# by.chromosome : if true, red is used for links within the same chromosome 
#    (we have removed those links already)
RCircos.Link.Plot(dfLinks, track.num = 3, by.chromosome = FALSE)

## 5. Save the final figure as a PDF.

In [ ]:
# define the file
pdf(file = "cancer_circos_r.pdf", height = 6, width = 6, compress = TRUE);

# These lines below are identical to the cell above
RCircos.Set.Core.Components(cyto.info = dfChrom, chr.exclude = NULL, tracks.inside = 4, tracks.outside = 0)
params <- RCircos.Get.Plot.Parameters()
params$track.background <- "white"
RCircos.Reset.Plot.Parameters(params)
RCircos.Set.Plot.Area()
RCircos.Chromosome.Ideogram.Plot() 
RCircos.Scatter.Plot(dfCNV, data.col = 4, track.num = 1, side = 'in', by.fold = 0.15, min.value = -1, max.value = 1)
RCircos.Scatter.Plot(dfBAF, data.col = 4, track.num = 2, side = 'in', min.value = 0, max.value = 1)
RCircos.Link.Plot(dfLinks, track.num = 3, by.chromosome = FALSE)

# close the graphics device
dev.off()

## Additional documentation links : 
- https://cran.r-project.org/web/packages/RCircos/vignettes/Using_RCircos.pdf, 
- https://cran.r-project.org/web/packages/RCircos/RCircos.pdf
- https://mrcuizhe.github.io/interacCircos_documentation/html/users_from_rcircos.html